In [16]:
import pandas as pd

rows = []

# sub-001 – sub-100  → PD = 1
for i in range(1, 101):
    rows.append({
        "subject": f"sub-{i:03d}",
        "group": 1
    })

# sub-101 – sub-149 → CTRL = 0
for i in range(101, 150):
    rows.append({
        "subject": f"sub-{i:03d}",
        "group": 0
    })

df = pd.DataFrame(rows)
df.to_csv("subject_groups.csv", index=False)

print("subject_groups.csv created")


subject_groups.csv created


In [23]:
import pandas as pd
import numpy as np
import os
from glob import glob

BASE = r"C:\Users\User\Documents\EEG_Project\rEEG"
GROUPS = pd.read_csv("subject_groups.csv")  
# columns: subject, group   (group: 1=PD, 0=CTRL)

features = []

for _, row in GROUPS.iterrows():
    subj = row['subject']
    grp = row['group']
    f = os.path.join(BASE, subj, "data", f"{subj}_bandpowers_epoch.csv")

    df = pd.read_csv(f)

    # average across epochs
    subj_vec = df.groupby("channel")["alpha2_rel"].mean().values

    features.append([subj_vec, grp])

X = np.stack([x[0] for x in features])
y = np.array([x[1] for x in features])

np.save("alpha2_rel_matrix.npy", X)
np.save("group_labels.npy", y)


In [24]:
import numpy as np
from scipy.stats import ttest_ind
from statsmodels.stats.multitest import multipletests
print("ok")

X = np.load("alpha2_rel_matrix.npy")
y = np.load("group_labels.npy")

pd = X[y == 1]
ctrl = X[y == 0]

pvals = np.zeros(X.shape[1])
for ch in range(X.shape[1]):
    _, pvals[ch] = ttest_ind(pd[:, ch], ctrl[:, ch], equal_var=False)

reject, p_fdr, _, _ = multipletests(pvals, method="fdr_bh")

print("Channels surviving FDR:", np.where(reject)[0])


ok
Channels surviving FDR: []


In [26]:
import numpy as np

# Load feature matrices
X = np.load("delta_rel_matrix.npy")   # shape: (149, n_channels)
y = np.load("group_labels.npy")       # shape: (149,)

pd = X[y==1]
ctrl = X[y==0]

# simple t-test to get pvals
from scipy.stats import ttest_ind
pvals = np.zeros(X.shape[1])
for ch in range(X.shape[1]):
    _, pvals[ch] = ttest_ind(pd[:, ch], ctrl[:, ch], equal_var=False)

# permutation test
def perm_test(x, y, n_perm=5000):
    obs = np.mean(x) - np.mean(y)
    comb = np.concatenate([x, y])
    cnt = 0
    for _ in range(n_perm):
        np.random.shuffle(comb)
        if abs(np.mean(comb[:len(x)]) - np.mean(comb[len(x):])) >= abs(obs):
            cnt += 1
    return (cnt + 1) / (n_perm + 1)  # avoids 0

best_ch = np.argmin(pvals)
p_perm = perm_test(pd[:, best_ch], ctrl[:, best_ch])
print("Permutation p:", p_perm)


Permutation p: 0.14917016596680663
